In [1]:
!pip install gradio
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, hamming_loss
import gradio as gr

     ---------------------------------------- 0.0/42.9 kB ? eta -:--:--
     ------------------------------------ - 41.0/42.9 kB 991.0 kB/s eta 0:00:01
     -------------------------------------- 42.9/42.9 kB 695.0 kB/s eta 0:00:00
   ---------------------------------------- 0.0/46.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/46.2 MB 2.0 MB/s eta 0:00:23
   ---------------------------------------- 0.1/46.2 MB 2.2 MB/s eta 0:00:22
   ---------------------------------------- 0.2/46.2 MB 1.5 MB/s eta 0:00:31
   ---------------------------------------- 0.2/46.2 MB 1.4 MB/s eta 0:00:34
   ---------------------------------------- 0.3/46.2 MB 1.3 MB/s eta 0:00:35
   ---------------------------------------- 0.4/46.2 MB 1.4 MB/s eta 0:00:33
   ---------------------------------------- 0.4/46.2 MB 1.4 MB/s eta 0:00:33
   ---------------------------------------- 0.5/46.2 MB 1.3 MB/s eta 0:00:35
   ---------------------------------------- 0.5/46.2 MB 1.3 MB/s eta 0:00:37
   --

In [5]:
df = pd.read_csv("food_ingredients_and_allergens.csv")

In [129]:
df.sample(10)

,Food Product,Main Ingredient,Sweetener,Fat/Oil,Seasoning,Allergens,Prediction,Allergen_Detected,Fish,Anchovies,...,Pork,Eggs,Oats,Coconut,Mustard,Ghee,Nuts,Dairy,Almonds,Soybeans
254,Biryani,Rice,None,Ghee,"Biryani masala, spices",None,Does not contain,0,0,0,...,0,0,0,0,0,0,0,0,0,0
50,Mushroom and Goat Cheese Flatbread,Mushrooms,None,Olive oil,"Goat cheese, herbs",Dairy,Contains,1,0,0,...,0,0,0,0,0,0,0,1,0,0
186,Quinoa Stuffed Peppers,Quinoa,None,Olive oil,"Vegetables, spices",None,Does not contain,0,0,0,...,0,0,0,0,0,0,0,0,0,0
240,Baked Chicken Wings,Chicken wings,None,Olive oil,Seasonings,None,Does not contain,0,0,0,...,0,0,0,0,0,0,0,0,0,0
192,Buffalo Wings,Chicken wings,None,Vegetable oil,Buffalo sauce,None,Does not contain,0,0,0,...,0,0,0,0,0,0,0,0,0,0
291,Lobster Bisque,Lobster,None,Butter,"Cream, brandy","Shellfish, Dairy",Contains,1,0,0,...,0,0,0,0,0,0,0,1,0,0
126,Strawberry Yogurt,"Yogurt (milk, cultures)",Sugar,None,Pectin,"Milk, Strawberries",Contains,1,0,0,...,0,0,0,0,0,0,0,0,0,0
277,Peanut Butter,Peanuts,Sugar,Vegetable oil,Salt,Peanuts,Contains,1,0,0,...,0,0,0,0,0,0,0,0,0,0
364,Pumpkin Pie,Pumpkin,Sugar,Butter,"Pumpkin spice, pastry","Wheat, Dairy",Contains,1,0,0,...,0,0,0,0,0,0,0,1,0,0
336,Garlic Bread,Bread,None,Butter,Garlic,"Wheat, Dairy",Contains,1,0,0,...,0,0,0,0,0,0,0,1,0,0


In [11]:
df.fillna("None", inplace=True)

In [13]:
Food_Ingredients = ['Food Product', 'Main Ingredient', 'Sweetener', 'Fat/Oil', 'Seasoning']

In [17]:
df_encoded = pd.get_dummies(df[Food_Ingredients])


In [19]:
df['Allergen_Detected'] = df['Allergens'].apply(lambda x: 1 if pd.notnull(x) and x != "None" else 0)
allergen_list = list(set([allergen.strip() for allergens in df['Allergens'].dropna() for allergen in allergens.split(',')]))
for allergen in allergen_list:
    df[allergen] = df['Allergens'].apply(lambda x: 1 if allergen in str(x) else 0)

In [23]:
X = df_encoded
y = df['Allergen_Detected'] 
y_multilabel = df[allergen_list] 

In [25]:
X_train , X_test, y_train , y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [27]:
_, _, y_train_multilabel, y_test_multilabel = train_test_split(X, y_multilabel, test_size=0.2, random_state=42)

#### Logistic Regression

In [36]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [46]:
print("Training Logistic Regression for Binary Classification...")
lr = LogisticRegression(max_iter=1000)

Training Logistic Regression for Binary Classification...


In [50]:
lr.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [54]:
y_pred = lr.predict(X_test)

In [56]:
binary_accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy (Binary Classification): {binary_accuracy * 100:.2f}%")
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy (Binary Classification): 86.25%

Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.78      0.77        23
           1       0.91      0.89      0.90        57

    accuracy                           0.86        80
   macro avg       0.83      0.84      0.83        80
weighted avg       0.86      0.86      0.86        80



In [75]:
from sklearn.multioutput import MultiOutputClassifier

# Train Logistic Regression for multilabel classification
print("\nTraining Logistic Regression for Multilabel Classification...")
lr_multilabel_model = MultiOutputClassifier(LogisticRegression(max_iter=1000))
lr_multilabel_model.fit(X_train, y_train_multilabel)

# Predict on the test set
y_pred_multilabel = lr_multilabel_model.predict(X_test)

# Evaluate the model
multilabel_accuracy = accuracy_score(y_test_multilabel, y_pred_multilabel)
print(f"Accuracy (Multilabel Classification): {multilabel_accuracy * 100:.2f}%")
print("\nClassification Report (for each label):\n")
for i, label in enumerate(y_multilabel.columns):
    print(f"{label}:\n{classification_report(y_test_multilabel.iloc[:, i], y_pred_multilabel[:, i])}")
    print("-" * 50)



Training Logistic Regression for Multilabel Classification...


ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 0

In [71]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Train Support Vector Machine for binary classification
print("Training Support Vector Machine (SVM) for Binary Classification...")
svm_model = SVC()
svm_model.fit(X_train, y_train)

# Predict on the test set
y_pred = svm_model.predict(X_test)

# Evaluate the model
binary_accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy (Binary Classification): {binary_accuracy * 100:.2f}%")
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Training Support Vector Machine (SVM) for Binary Classification...
Accuracy (Binary Classification): 88.75%

Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.74      0.79        23
           1       0.90      0.95      0.92        57

    accuracy                           0.89        80
   macro avg       0.88      0.84      0.86        80
weighted avg       0.89      0.89      0.89        80



In [73]:
from sklearn.multioutput import MultiOutputClassifier

# Train Support Vector Machine for multilabel classification
print("\nTraining Support Vector Machine (SVM) for Multilabel Classification...")
svm_multilabel_model = MultiOutputClassifier(SVC())
svm_multilabel_model.fit(X_train, y_train_multilabel)

# Predict on the test set
y_pred_multilabel = svm_multilabel_model.predict(X_test)

# Evaluate the model
multilabel_accuracy = accuracy_score(y_test_multilabel, y_pred_multilabel)
print(f"Accuracy (Multilabel Classification): {multilabel_accuracy * 100:.2f}%")
print("\nClassification Report (for each label):\n")
for i, label in enumerate(y_multilabel.columns):
    print(f"{label}:\n{classification_report(y_test_multilabel.iloc[:, i], y_pred_multilabel[:, i])}")
    print("-" * 50)



Training Support Vector Machine (SVM) for Multilabel Classification...


ValueError: The number of classes has to be greater than one; got 1 class

#### Random Forest Classifier

In [59]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [77]:
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [79]:
y_pred = rf.predict(X_test)

In [67]:
binary_accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy (Binary Classification): {binary_accuracy * 100:.2f}%")
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy (Binary Classification): 85.00%

Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.61      0.70        23
           1       0.86      0.95      0.90        57

    accuracy                           0.85        80
   macro avg       0.84      0.78      0.80        80
weighted avg       0.85      0.85      0.84        80



In [81]:
from sklearn.multioutput import MultiOutputClassifier

multi_label_model = MultiOutputClassifier(RandomForestClassifier(random_state=42))
multi_label_model.fit(X_train, y_train_multilabel)

MultiOutputClassifier(estimator=RandomForestClassifier(random_state=42))

In [85]:
y_pred_binary = rf.predict(X_test)
binary_accuracy = accuracy_score(y_test, y_pred_binary)

In [87]:
y_pred_multilabel = multi_label_model.predict(X_test)
multi_label_accuracy = 1 - hamming_loss(y_test_multilabel, y_pred_multilabel)

In [89]:
print(f"Binary Classification Accuracy (Contains Allergen or not): {binary_accuracy:.2f}")
print(f"Multi-label Classification Accuracy (Specific Allergens): {multi_label_accuracy:.2f}")

Binary Classification Accuracy (Contains Allergen or not): 0.86
Multi-label Classification Accuracy (Specific Allergens): 0.98


In [99]:
def allergens_detection(food_product, main_ingredient, sweetener, fat_oil, seasoning):
    # Prepare input data for prediction
    input_data = {
        'Food Product': food_product.title(),
        'Main Ingredient': main_ingredient.title(),
        'Sweetener': sweetener.capitalize(),
        'Fat/Oil': fat_oil.capitalize(),
        'Seasoning': seasoning.capitalize()
    }

    # Convert the input data to a DataFrame
    input_df = pd.DataFrame([input_data])

    # One-hot encode the input data
    input_encoded = pd.get_dummies(input_df)

    # Align the encoded input columns with training data columns
    input_encoded = input_encoded.reindex(columns=X.columns, fill_value=0)

    # Predict if allergens are present
    allergen_present = rf.predict(input_encoded)[0]  # `rf` is the Random Forest model
    if allergen_present == 0:
        return "No allergens present."

    # Predict specific allergens
    specific_allergens = multi_label_model.predict(input_encoded)  # `multi_label_model` is the multilabel classifier
    allergens_detected = [
        allergen for allergen, present in zip(allergen_list, specific_allergens[0]) if present
    ]

    # Return results
    if allergens_detected:
        return f"Allergens present: {', '.join(allergens_detected)}"
    else:
        return "No specific allergens identified."


In [101]:
print(allergens_detection(
    food_product="Chocolate cake",
    main_ingredient="flour",
    sweetener="Sugar",
    fat_oil="Butter",
    seasoning="Cinnamon, Nutmeg"
))


Allergens present: Wheat, Dairy


In [105]:
print(allergens_detection(
    food_product="Butter Chicken",
    main_ingredient="Chicken",
    sweetener="None",
    fat_oil="Butter",
    seasoning="Spices, Cream"
))



Allergens present: Dairy


In [107]:
print(allergens_detection(
    food_product="Samosa",
    main_ingredient="Potatoes",
    sweetener="None",
    fat_oil="Vegetable oil",
    seasoning="Wheat flour, Cumin"
))


No allergens present.


In [109]:
print(allergens_detection(
    food_product="Rasgulla",
    main_ingredient="Milk solids",
    sweetener="Sugar",
    fat_oil="None",
    seasoning="Cardamom"
))


Allergens present: Dairy


In [111]:
print(allergens_detection(
    food_product="Paneer Tikka",
    main_ingredient="Paneer",
    sweetener="None",
    fat_oil="Ghee",
    seasoning="Spices, Lemon juice"
))


Allergens present: Dairy


In [113]:
print(allergens_detection(
    food_product="Gulab Jamun",
    main_ingredient="Milk solids",
    sweetener="Sugar syrup",
    fat_oil="Ghee",
    seasoning="Cardamom"
))


No specific allergens identified.


In [115]:
print(allergens_detection(
    food_product="Butter Naan",
    main_ingredient="Flour",
    sweetener="None",
    fat_oil="Butter",
    seasoning="None"
))


Allergens present: Wheat, Dairy


In [117]:
print(allergens_detection(
    food_product="Chicken Biryani",
    main_ingredient="Chicken",
    sweetener="None",
    fat_oil="Ghee",
    seasoning="Basmati rice, spices"
))


No allergens present.


In [119]:
print(allergens_detection(
    food_product="Gulab Jamun",
    main_ingredient="Milk solids",
    sweetener="Sugar",
    fat_oil="Ghee",
    seasoning="Cardamom syrup"
))


Allergens present: Dairy


In [137]:
print(allergens_detection(
    food_product="Strawberry Yogurt",
    main_ingredient="Yogurt (milk, cultures)",
    sweetener="Sugar",
    fat_oil="None",
    seasoning="Pectin	Milk, Strawberries"
))


No specific allergens identified.


In [ ]:
Strawberry Yogurt	Yogurt (milk, cultures)	Sugar	None	Pectin	Milk, Strawberries	